optuna 써보기

In [8]:
import pandas as pd

from sklearn.datasets import load_iris

In [9]:
import optuna

# get data

In [10]:
iris = load_iris()
iris.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [11]:
iris_df = pd.DataFrame(iris['data'], columns=iris['feature_names'])
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


# split data

In [12]:
from sklearn.model_selection import train_test_split

In [14]:
train_x, test_x, train_y, test_y = train_test_split(iris_df, iris['target'], test_size=.2)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((120, 4), (30, 4), (120,), (30,))

# mlpregressor

In [15]:
mname = 'mlpregressor'

In [17]:
from sklearn.neural_network import MLPRegressor
# from sklearn.model_selection import RandomizedSearchCV

In [18]:
params = {
    'hidden_layer_sizes': [10, 50, 100, 200],
    'activation' : ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': [1, 0.1, 0.01, 0.001, 0.0001],
    'batch_size': [10, 50, 100, 200, 500, 1000],
    'warm_start': [True, False],
    'learning_rate_init': [1, 0.1, 0.01, 0.001, 0.0001], #24 added
    'max_iter': [100, 200, 500, 1000, 5000], #24 added
    
}

In [19]:
model = MLPRegressor()
model.fit(train_x, train_y)

MLPRegressor()

In [23]:
from sklearn.metrics import mean_squared_error, r2_score

In [24]:
# predict
train_pred = model.predict(train_x)
test_pred = model.predict(test_x)

# evaluate
train_rmse = mean_squared_error(train_y, train_pred, squared=False)
test_rmse = mean_squared_error(test_y, test_pred, squared=False)
train_r2 = r2_score(train_y, train_pred)
test_r2 = r2_score(test_y, test_pred)

print('train_rmse: ', train_rmse)
print('test_rmse: ', test_rmse)
print('train_r2: ', train_r2)
print('test_r2: ', test_r2)

train_rmse:  0.261609533010679
test_rmse:  0.2757712430066753
train_r2:  0.8985976450484882
test_r2:  0.8797103679745131


In [28]:
import sklearn.ensemble
import sklearn.model_selection
import sklearn.svm

# FYI: Objective functions can take additional arguments
# (https://optuna.readthedocs.io/en/stable/faq.html#objective-func-additional-args).
def objective(trial):
    iris = sklearn.datasets.load_iris()
    x, y = iris.data, iris.target

    # classifier_name = trial.suggest_categorical("classifier", ["SVC", "RandomForest"])
    # if classifier_name == "SVC":
    #     svc_c = trial.suggest_float("svc_c", 1e-10, 1e10, log=True)
    #     classifier_obj = sklearn.svm.SVC(C=svc_c, gamma="auto")
    # else:
    rf_max_depth = trial.suggest_int("rf_max_depth", 2, 32, log=True)
    classifier_obj = sklearn.ensemble.RandomForestClassifier(
        max_depth=rf_max_depth, n_estimators=10
    )

    score = sklearn.model_selection.cross_val_score(classifier_obj, x, y, n_jobs=-1, cv=3)
    accuracy = score.mean()
    return accuracy


# if __name__ == "__main__":
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)
print(study.best_trial)

[I 2023-04-25 00:21:13,975] A new study created in memory with name: no-name-58ffa261-9a6a-4415-83ba-24a6567d30ab
[I 2023-04-25 00:21:14,037] Trial 0 finished with value: 0.9533333333333333 and parameters: {'rf_max_depth': 14}. Best is trial 0 with value: 0.9533333333333333.
[I 2023-04-25 00:21:14,067] Trial 1 finished with value: 0.9533333333333333 and parameters: {'rf_max_depth': 4}. Best is trial 0 with value: 0.9533333333333333.
[I 2023-04-25 00:21:14,093] Trial 2 finished with value: 0.96 and parameters: {'rf_max_depth': 21}. Best is trial 2 with value: 0.96.
[I 2023-04-25 00:21:14,124] Trial 3 finished with value: 0.96 and parameters: {'rf_max_depth': 12}. Best is trial 2 with value: 0.96.
[I 2023-04-25 00:21:14,168] Trial 4 finished with value: 0.9666666666666667 and parameters: {'rf_max_depth': 8}. Best is trial 4 with value: 0.9666666666666667.
[I 2023-04-25 00:21:14,198] Trial 5 finished with value: 0.9733333333333333 and parameters: {'rf_max_depth': 3}. Best is trial 5 with 

FrozenTrial(number=5, state=TrialState.COMPLETE, values=[0.9733333333333333], datetime_start=datetime.datetime(2023, 4, 25, 0, 21, 14, 170664), datetime_complete=datetime.datetime(2023, 4, 25, 0, 21, 14, 198800), params={'rf_max_depth': 3}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'rf_max_depth': IntDistribution(high=32, log=True, low=2, step=1)}, trial_id=5, value=None)
